#### Imports

In [1]:
import cv2
from pathlib import Path
from omegaconf import OmegaConf
import torch
import numpy as np
import wandb
from hydra import initialize, compose
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torch import nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms
import wandb
from datetime import datetime
import matplotlib.pyplot as plt 


### Se toman las 4 carpetas (cable, capsule, screw y transistor) y se separa su información de testing training y se juntan en un solo dataset, igualmente guardando las etiquetas y se setea el tamaño de cada imagen en 128x128 como se indica en el documento

In [ ]:
# Configuración
DATASETS = ['cable', 'capsule', 'screw', 'transistor']
BASE_PATH = Path('../TareaAutoEncoders')
OUTPUT_PATH = BASE_PATH / 'DATASET_128x128'
IMAGE_SIZE = (128, 128)

# Crear estructura de salida (carpetas planas)
for split in ['train', 'test', 'ground_truth']:
    (OUTPUT_PATH / split).mkdir(parents=True, exist_ok=True)

def process_and_save(src_path: Path, dest_dir: Path, prefix: str, is_mask=False):
    """Lee, redimensiona y guarda. Si is_mask usa INTER_NEAREST."""
    try:
        img = cv2.imread(str(src_path), cv2.IMREAD_UNCHANGED)
        if img is None:
            print(f"No se pudo leer: {src_path}")
            return False
        interp = cv2.INTER_NEAREST if is_mask else cv2.INTER_AREA
        resized = cv2.resize(img, IMAGE_SIZE, interpolation=interp)
        dest = dest_dir / f"{prefix}_{src_path.stem}{src_path.suffix}"
        cv2.imwrite(str(dest), resized)
        return True
    except Exception as e:
        print(f"Error con {src_path}: {e}")
        return False

# Procesar datasets
for dataset in DATASETS:
    base = BASE_PATH / dataset

    # train -> normalmente sólo 'good' en estos datasets
    train_dir = base / 'train'
    if train_dir.exists():
        for cls in train_dir.iterdir():
            if not cls.is_dir(): 
                continue
            for img in cls.glob('*.*'):
                prefix = f"{dataset}_train_{cls.name}"
                process_and_save(img, OUTPUT_PATH / 'train', prefix, is_mask=False)

    # test -> incluir good y defectos
    test_dir = base / 'test'
    if test_dir.exists():
        for cls in test_dir.iterdir():
            if not cls.is_dir():
                continue
            for img in cls.glob('*.*'):
                prefix = f"{dataset}_test_{cls.name}"
                process_and_save(img, OUTPUT_PATH / 'test', prefix, is_mask=False)

    # ground_truth -> máscaras (usar nearest)
    gt_dir = base / 'ground_truth'
    if gt_dir.exists():
        for cls in gt_dir.iterdir():
            if not cls.is_dir():
                continue
            for img in cls.glob('*.*'):
                prefix = f"{dataset}_gt_{cls.name}"
                process_and_save(img, OUTPUT_PATH / 'ground_truth', prefix, is_mask=True)


## Configuración de los archivos Hydra

### ¿Por qué?
La librería Hydra permite establecer las configuraciones que va a tener la ejecución del modelo, lo cual permite una forma eficaz de cambiar los parámetros con los que será entrenado el mismo sin tener que hacer variaciones en los parámetros del código

In [2]:
# Crear estructura base
conf_path = Path("conf")
conf_path.mkdir(exist_ok=True)

print("Directorio conf/ creado")

Directorio conf/ creado


In [3]:
# Celda 2: Crear carpetas necesarias
subdirs = ["model", "trainer", "logger", "loss", "optimizer"]
for subdir in subdirs:
    (conf_path / subdir).mkdir(exist_ok=True)

print("Subdirectorios creados:")
for subdir in subdirs:
    print(f"   - conf/{subdir}/")

Subdirectorios creados:
   - conf/model/
   - conf/trainer/
   - conf/logger/
   - conf/loss/
   - conf/optimizer/


### Solicitudes del enunciado:

A como estaba solicitado en el enunciado, se crean diferentes sets de configuraciones .yaml que serán guardadas en la carpeta conf para su posterior uso


In [4]:
# Celda: Crear variaciones de configuración para experimentos
# Variación 1: Latent dim pequeño
latent_small_yaml = """name: autoencoder_latent_small
in_channels: 3
hidden_dims: [32, 64, 128, 256]
latent_dim: 128
use_batch_norm: true
dropout_rate: 0.0
"""

with open("conf/model/autoencoder_latent_small.yaml", "w") as f:
    f.write(latent_small_yaml)

# Variación 2: Latent dim grande
latent_large_yaml = """name: autoencoder_latent_large
in_channels: 3
hidden_dims: [32, 64, 128, 256]
latent_dim: 1024
use_batch_norm: true
dropout_rate: 0.0
"""

with open("conf/model/autoencoder_latent_large.yaml", "w") as f:
    f.write(latent_large_yaml)

print("Variaciones de modelo creadas:")
print("   - autoencoder_latent_small (128)")
print("   - autoenceoder_latent_large (1024)")

Variaciones de modelo creadas:
   - autoencoder_latent_small (128)
   - autoenceoder_latent_large (1024)


In [5]:
# Crear conf/config.yaml (configuración principal)
config_yaml = """defaults:
  - model: autoencoder
  - trainer: default
  - logger: wandb
  - loss: l2
  - optimizer: adam_mid

seed: 42

data:
  data_dir: 'DATASET_128x128'
  image_size: 128
  batch_size: 32
  num_workers: 0
  validation_split: 0.15
  test_split: 0.15

callbacks:
  monitor: "val/loss"
  mode: "min"
  filename: "{epoch:02d}-{val/loss:.4f}"
  save_top_k: 3

experiment:
  name: "default_experiment"
  description: "Default autoencoder experiment"
"""

with open("conf/config.yaml", "w") as f:
    f.write(config_yaml)

print("conf/config.yaml creado")

conf/config.yaml creado


In [6]:
# Crear modelos - conf/model/autoencoder.yaml
autoencoder_yaml = """name: autoencoder
in_channels: 3
hidden_dims: [32, 64, 128, 256]
latent_dim: 512
use_batch_norm: true
dropout_rate: 0.0
"""

with open("conf/model/autoencoder.yaml", "w") as f:
    f.write(autoencoder_yaml)

print("conf/model/autoencoder.yaml creado")

conf/model/autoencoder.yaml creado


In [7]:
# Crear modelos - conf/model/unet.yaml
unet_yaml = """name: unet
in_channels: 3
base_channels: 32
depth: 4
use_batch_norm: true
dropout_rate: 0.0
"""

with open("conf/model/unet.yaml", "w") as f:
    f.write(unet_yaml)

print("conf/model/unet.yaml creado")

conf/model/unet.yaml creado


In [8]:
# Variaciones de autoencoder con latent_dim pequeño
autoencoder_small_yaml = """name: autoencoder_small
in_channels: 3
hidden_dims: [32, 64, 128]
latent_dim: 128
use_batch_norm: true
dropout_rate: 0.0
"""

with open("conf/model/autoencoder_small.yaml", "w") as f:
    f.write(autoencoder_small_yaml)

print("conf/model/autoencoder_small.yaml creado (latent_dim: 128)")

conf/model/autoencoder_small.yaml creado (latent_dim: 128)


In [9]:
# Variaciones de autoencoder con latent_dim grande
autoencoder_large_yaml = """name: autoencoder_large
in_channels: 3
hidden_dims: [32, 64, 128, 256, 512]
latent_dim: 1024
use_batch_norm: true
dropout_rate: 0.1
"""

with open("conf/model/autoencoder_large.yaml", "w") as f:
    f.write(autoencoder_large_yaml)

print("conf/model/autoencoder_large.yaml creado (latent_dim: 1024)")

conf/model/autoencoder_large.yaml creado (latent_dim: 1024)


In [10]:
# Funciones de pérdida - L1
l1_yaml = """name: l1
type: L1Loss
weight: 1.0
"""

with open("conf/loss/l1.yaml", "w") as f:
    f.write(l1_yaml)

print("conf/loss/l1.yaml creado")

conf/loss/l1.yaml creado


In [11]:
# Funciones de pérdida - L2 (MSE)
l2_yaml = """name: l2
type: MSELoss
weight: 1.0
"""

with open("conf/loss/l2.yaml", "w") as f:
    f.write(l2_yaml)

print("conf/loss/l2.yaml creado")

conf/loss/l2.yaml creado


In [12]:
# Funciones de pérdida - SSIM
ssim_yaml = """name: ssim
type: SSIMLoss
weight: 1.0
window_size: 11
sigma: 1.5
data_range: 1.0
"""

with open("conf/loss/ssim.yaml", "w") as f:
    f.write(ssim_yaml)

print("conf/loss/ssim.yaml creado")

conf/loss/ssim.yaml creado


In [13]:
# Funciones de pérdida - SSIM + L1
ssim_l1_yaml = """name: ssim_l1
type: SSIMLoss_L1
weight_ssim: 0.5
weight_l1: 0.5
window_size: 11
sigma: 1.5
data_range: 1.0
"""

with open("conf/loss/ssim_l1.yaml", "w") as f:
    f.write(ssim_l1_yaml)

print("conf/loss/ssim_l1.yaml creado")

conf/loss/ssim_l1.yaml creado


In [14]:
# Trainer - conf/trainer/default.yaml
trainer_yaml = """max_epochs: 20
gpus: 1
precision: 32
deterministic: true
check_val_every_n_epoch: 1
log_every_n_steps: 10
enable_model_summary: true
gradient_clip_val: 0.0
enable_progress_bar: true
"""

with open("conf/trainer/default.yaml", "w") as f:
    f.write(trainer_yaml)

print("conf/trainer/default.yaml creado")

conf/trainer/default.yaml creado


In [15]:
# Logger - conf/logger/wandb.yaml
wandb_yaml = """project: ae_experiments
entity: null
log_model: false
offline: false
tags: []
"""

with open("conf/logger/wandb.yaml", "w") as f:
    f.write(wandb_yaml)

print("conf/logger/wandb.yaml creado")

conf/logger/wandb.yaml creado


In [16]:
# Optimizer - Adam con LR bajo
adam_low_yaml = """name: adam_low
type: Adam
lr: 1e-4
weight_decay: 0.0
betas: [0.9, 0.999]
"""

with open("conf/optimizer/adam_low.yaml", "w") as f:
    f.write(adam_low_yaml)

print("conf/optimizer/adam_low.yaml creado (lr: 1e-4)")

conf/optimizer/adam_low.yaml creado (lr: 1e-4)


In [17]:
# Optimizer - Adam con LR medio
adam_mid_yaml = """name: adam_mid
type: Adam
lr: 1e-3
weight_decay: 0.0
betas: [0.9, 0.999]
"""

with open("conf/optimizer/adam_mid.yaml", "w") as f:
    f.write(adam_mid_yaml)

print("conf/optimizer/adam_mid.yaml creado (lr: 1e-3)")

conf/optimizer/adam_mid.yaml creado (lr: 1e-3)


In [18]:
# Optimizer - Adam con LR alto
adam_high_yaml = """name: adam_high
type: Adam
lr: 5e-3
weight_decay: 1e-5
betas: [0.9, 0.999]
"""

with open("conf/optimizer/adam_high.yaml", "w") as f:
    f.write(adam_high_yaml)

print("conf/optimizer/adam_high.yaml creado (lr: 5e-3)")

conf/optimizer/adam_high.yaml creado (lr: 5e-3)


## Definición del DataModule y modelo base (Autoencoder clásico con Hydra + PyTorch Lightning)


### Definición del Dataset para DATASET_128x128

En esta sección definimos una clase `MVTecDataset` basada en `torch.utils.data.Dataset`
que carga las imágenes ya preprocesadas a tamaño **128×128**.

Las imágenes se cargan en formato RGB y se convierten a tensores normalizados en \[0, 1].
Este dataset se utilizará dentro del `LightningDataModule` para separar train/val/test.


In [19]:
class MVTecDataset(Dataset):

    def __init__(self, root_dir, split="train", transform=None):
        super().__init__()
        self.root_dir = Path(root_dir)
        self.split = split
        self.transform = transform

        split_dir = self.root_dir / split
        exts = (".png", ".jpg", ".jpeg")

        self.image_paths = sorted(
            [p for p in split_dir.glob("*.*") if p.suffix.lower() in exts],
            key=lambda p: p.name
        )

        if len(self.image_paths) == 0:
            print(f"[WARNING] No se encontraron imágenes en {split_dir}")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert("RGB")

        if self.transform is not None:
            img = self.transform(img)

        # Para autoencoder solo necesitamos la imagen (entrada = salida)
        return img


### LightningDataModule para MVTec

Para estructurar el flujo de datos usando PyTorch Lightning, se define un
`LightningDataModule` llamado `MVTecDataModule`.

Este módulo:

- Recibe los hiperparámetros desde la configuración (`cfg.data`):
  - `data_dir`, `batch_size`, `num_workers`, `validation_split`.
- Construye el `Dataset` de entrenamiento completo y lo separa en:
  - subconjunto de **train**
  - subconjunto de **validation** (usando `validation_split`).
- Crea el `Dataset` de **test**.
- Expone los `DataLoader`:
  - `train_dataloader()`
  - `val_dataloader()`
  - `test_dataloader()`

De esta forma, el mismo `DataModule` se reutiliza para todos los modelos y
experimentos (distintas funciones de pérdida, arquitecturas, etc.).


In [20]:
class MVTecDataModule(pl.LightningDataModule):
    def __init__(
        self,
        data_dir,
        batch_size=32,
        num_workers=2,
        val_split=0.15,
    ):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.val_split = val_split

        # Transformación básica: convertir a tensor en [0,1]
        self.transform = transforms.ToTensor()

    def setup(self, stage=None):
        # Dataset completo de entrenamiento (después se divide en train/val)
        full_train = MVTecDataset(
            root_dir=self.data_dir,
            split="train",
            transform=self.transform,
        )

        n_total = len(full_train)
        n_val = int(self.val_split * n_total)
        n_train = n_total - n_val

        self.train_set, self.val_set = torch.utils.data.random_split(
            full_train,
            [n_train, n_val],
            generator=torch.Generator().manual_seed(42),
        )

        # Dataset de test
        self.test_set = MVTecDataset(
            root_dir=self.data_dir,
            split="test",
            transform=self.transform,
        )

    def train_dataloader(self):
        return DataLoader(
            self.train_set,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_set,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_set,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
        )


### Construcción de función de pérdida y optimizador desde Hydra

La configuración de la tarea se maneja con Hydra. En particular:

- `conf/loss/*.yaml` define el tipo de función de pérdida a usar
  (L1, L2, SSIM, SSIM + L1).
- `conf/optimizer/*.yaml` define el tipo de optimizador y sus hiperparámetros
  (por ejemplo, Adam con distintas tasas de aprendizaje).

Para desacoplar el modelo de estas decisiones, se implementan dos funciones:

- `build_loss(loss_cfg)`: a partir de `cfg.loss` devuelve un objeto de pérdida
  de PyTorch.
- `build_optimizer(optimizer_cfg, parameters)`: a partir de `cfg.optimizer`
  devuelve una instancia del optimizador apropiado.



In [21]:
def build_loss(loss_cfg): #Loss_cfg es el diccionario generado por Hydra a partir de las configuraciones .yaml
    """
    Construye la función de pérdida a partir de cfg.loss.
    Por ahora se implementan L1 y L2 (MSE).
    """
    loss_type = loss_cfg.type # Se asignan los valores ajustados a la configuracion

    if loss_type == "L1Loss":
        return nn.L1Loss()
    elif loss_type == "MSELoss":
        return nn.MSELoss()
    else:
        # Aquí luego se agregarán SSIM y SSIM+L1
        raise NotImplementedError(f"Pérdida '{loss_type}' aún no implementada en este notebook.")
    

def build_optimizer(optimizer_cfg, parameters):
    """
    Construye el optimizador a partir de cfg.optimizer.
    Actualmente soporta Adam con lr configurable.
    """
    opt_type = optimizer_cfg.type

    if opt_type == "Adam":
        return torch.optim.Adam(
            parameters,
            lr=optimizer_cfg.lr,
            weight_decay=optimizer_cfg.weight_decay,
            betas=tuple(optimizer_cfg.betas),
        )
    else:
        raise NotImplementedError(f"Optimizer '{opt_type}' no implementado.")


### Modelo base: Autoencoder clásico con PyTorch Lightning

En esta sección se define el modelo de **autoencoder clásico** como un
`LightningModule` llamado `LitAutoencoder`.

Este módulo:

- Lee su configuración desde `cfg.model`:
  - `in_channels`
  - `hidden_dims` (lista de canales intermedios)
  - `latent_dim`
- Construye un **encoder** convolucional que reduce la resolución de la imagen.
- Aplica capas totalmente conectadas para:
  - Proyectar la salida del encoder a un espacio latente de dimensión `latent_dim`.
  - Reconstruir desde el espacio latente a la forma intermedia del decoder.
- Construye un **decoder** con convoluciones transpuestas para recuperar
  una imagen de tamaño 128×128 y 3 canales.
- Utiliza la función de pérdida definida en `cfg.loss`.
- Utiliza el optimizador definido en `cfg.optimizer`.

Este modelo será el primero en usarse para los experimentos de la tarea
(con distintas funciones de pérdida). 


In [22]:
class LitAutoencoder(pl.LightningModule):
    def __init__(self, model_cfg, loss_cfg, optimizer_cfg, image_size=128):
        super().__init__()
        # Guardamos la config de modelo como hyperparams (para reproducibilidad)
        self.save_hyperparameters(OmegaConf.to_container(model_cfg, resolve=True))

        self.model_cfg = model_cfg
        self.loss_cfg = loss_cfg
        self.optimizer_cfg = optimizer_cfg
        self.image_size = image_size

        in_channels = model_cfg.in_channels
        hidden_dims = list(model_cfg.hidden_dims)
        latent_dim = model_cfg.latent_dim

        # Encoder: secuencia de convoluciones con stride 2
        modules = []
        channels = in_channels
        size = image_size
        for h in hidden_dims:
            modules.append(nn.Conv2d(channels, h, kernel_size=3, stride=2, padding=1))
            modules.append(nn.ReLU())
            channels = h
            size = size // 2  # cada conv con stride 2 reduce la mitad

        self.encoder = nn.Sequential(*modules)
        self.enc_out_channels = channels
        self.enc_out_size = size

        self.flatten = nn.Flatten()
        self.fc_mu = nn.Linear(channels * size * size, latent_dim)
        self.fc_decode = nn.Linear(latent_dim, channels * size * size)

        # Decoder: conv transpuestas para volver a 3x128x128
        modules = []
        hidden_dims_rev = list(hidden_dims[::-1])

        for i in range(len(hidden_dims_rev) - 1):
            modules.append(
                nn.ConvTranspose2d(
                    hidden_dims_rev[i],
                    hidden_dims_rev[i + 1],
                    kernel_size=4,
                    stride=2,
                    padding=1,
                )
            )
            modules.append(nn.ReLU())

        modules.append(
            nn.ConvTranspose2d(
                hidden_dims_rev[-1],
                in_channels,
                kernel_size=4,
                stride=2,
                padding=1,
            )
        )
        modules.append(nn.Sigmoid())  # salida en [0,1]

        self.decoder = nn.Sequential(*modules)

        # Pérdida
        self.criterion = build_loss(loss_cfg)

    def forward(self, x):
        x_enc = self.encoder(x)
        x_flat = self.flatten(x_enc)
        z = self.fc_mu(x_flat)
        x_dec_flat = self.fc_decode(z)
        x_dec = x_dec_flat.view(
            x.shape[0],
            self.enc_out_channels,
            self.enc_out_size,
            self.enc_out_size,
        )
        x_hat = self.decoder(x_dec)
        return x_hat

    def _shared_step(self, batch, stage):
        x = batch
        x_hat = self(x)
        loss = self.criterion(x_hat, x)
        self.log(f"{stage}_loss", loss, prog_bar=True)
        return loss

    def training_step(self, batch, batch_idx):
        return self._shared_step(batch, "train")

    def validation_step(self, batch, batch_idx):
        self._shared_step(batch, "val")

    def test_step(self, batch, batch_idx):
        self._shared_step(batch, "test")

    def configure_optimizers(self):
        optimizer = build_optimizer(self.optimizer_cfg, self.parameters())
        return optimizer


### Función principal de entrenamiento con Hydra y WandB

Finalmente, se define una función `train_autoencoder_with_hydra()` que
integra todos los componentes anteriores:

1. Inicializa Hydra y carga la configuración desde `conf/config.yaml`.
2. Construye el `MVTecDataModule` usando `cfg.data`.
3. Construye el `LitAutoencoder` usando:
   - `cfg.model` (arquitectura del autoencoder clásico),
   - `cfg.loss` (función de pérdida),
   - `cfg.optimizer` (optimizador).
4. Inicializa un `WandbLogger` con los parámetros de `cfg.logger`.
5. Crea un `Trainer` de PyTorch Lightning con los parámetros definidos en `cfg.trainer`.
6. Llama a `trainer.fit(model, datamodule=dm)` para entrenar el modelo.



In [23]:
def train_autoencoder_with_hydra():
    """
    Función de entrenamiento principal.
    Usa Hydra para cargar conf/config.yaml y los subarchivos.
    """
    with initialize(config_path="conf", version_base=None):
        cfg = compose(config_name="config")

    print("Configuración cargada:")
    print(OmegaConf.to_yaml(cfg))

    # DataModule
    dm = MVTecDataModule(
        data_dir=cfg.data.data_dir,
        batch_size=cfg.data.batch_size,
        num_workers=cfg.data.num_workers,
        val_split=cfg.data.validation_split,
    )

    # Modelo (autoencoder clásico)
    model = LitAutoencoder(
        model_cfg=cfg.model,
        loss_cfg=cfg.loss,
        optimizer_cfg=cfg.optimizer,
        image_size=cfg.data.image_size,
    )

    # Logger de WandB
    wandb_logger = WandbLogger(
        project=cfg.logger.project,
        entity=cfg.logger.entity,
        log_model=cfg.logger.log_model,
    )

    # Trainer a partir de cfg.trainer
    trainer = pl.Trainer(
        max_epochs=cfg.trainer.max_epochs,
        log_every_n_steps=cfg.trainer.log_every_n_steps,
        deterministic=cfg.trainer.deterministic,
        enable_model_summary=cfg.trainer.enable_model_summary,
        enable_progress_bar=cfg.trainer.enable_progress_bar,
        logger=wandb_logger,
    )

    trainer.fit(model, datamodule=dm)


### Configuraciones de funciones de pérdida (Hydra)

Se requiere evaluar distintas funciones de pérdida para el autoencoder:

- **L1**  
- **L2 (MSE)**  
- **SSIM**  
- **SSIM + L1**

Para permitir cambiar entre estas variantes desde Hydra sin modificar código,
se definen cuatro archivos de configuración en `conf/loss/`:

- `l1.yaml`
- `l2.yaml`
- `ssim.yaml`
- `ssim_l1.yaml`

Cada uno especifica el tipo de pérdida a usar (`type`) y, en el caso de SSIM,
algunos hiperparámetros adicionales.


In [24]:
Path("conf/loss").mkdir(parents=True, exist_ok=True)

l1_yaml = """type: L1Loss
name: "L1"
"""

l2_yaml = """type: MSELoss
name: "L2"
"""

ssim_yaml = """type: SSIM
name: "SSIM"
window_size: 11
sigma: 1.5
data_range: 1.0
"""

ssim_l1_yaml = """type: SSIM_L1
name: "SSIM+L1"
window_size: 11
sigma: 1.5
data_range: 1.0
l1_weight: 0.1
"""

with open("conf/loss/l1.yaml", "w") as f:
    f.write(l1_yaml)

with open("conf/loss/l2.yaml", "w") as f:
    f.write(l2_yaml)

with open("conf/loss/ssim.yaml", "w") as f:
    f.write(ssim_yaml)

with open("conf/loss/ssim_l1.yaml", "w") as f:
    f.write(ssim_l1_yaml)

print("Archivos de configuración de pérdidas creados/actualizados en conf/loss/")


Archivos de configuración de pérdidas creados/actualizados en conf/loss/


### Implementación de la pérdida SSIM

La métrica **Structural Similarity Index (SSIM)** mide la similitud estructural
entre dos imágenes. A diferencia de L1/L2, que comparan píxel a píxel,
SSIM toma en cuenta:

- luminancia,
- contraste,
- estructura local.

Para usar SSIM como pérdida, se suele minimizar `1 - SSIM(x, y)`, donde `x` es
la imagen original y `y` la reconstrucción del autoencoder.

A continuación se define una implementación en PyTorch que:

- Convierte la fórmula de SSIM a operaciones de convolución 2D con un kernel
  gaussiano.
- Calcula SSIM de forma local y luego promedia el resultado.
- Devuelve `1 - SSIM` como valor de pérdida.


In [25]:

class SSIMLoss(nn.Module):
    def __init__(
        self,
        window_size=11,
        sigma=1.5,
        data_range=1.0,
        channel=3,
        K1=0.01,
        K2=0.03,
    ):
        """
        Implementación de SSIM como pérdida: loss = 1 - SSIM.
        Asume imágenes en rango [0, data_range] y 3 canales por defecto.
        """
        super().__init__()
        self.window_size = window_size
        self.sigma = sigma
        self.data_range = data_range
        self.channel = channel
        self.K1 = K1
        self.K2 = K2

        self.register_buffer("window", self._create_window(window_size, sigma, channel))

    def _gaussian(self, window_size, sigma):
        gauss = torch.tensor(
            [
                (-(x - window_size // 2) ** 2) / float(2 * sigma**2)
                for x in range(window_size)
            ]
        )
        gauss = torch.exp(gauss)
        return gauss / gauss.sum()

    def _create_window(self, window_size, sigma, channel):
        _1d_window = self._gaussian(window_size, sigma).unsqueeze(1)
        _2d_window = _1d_window @ _1d_window.t()  # producto externo
        _2d_window = _2d_window.float().unsqueeze(0).unsqueeze(0)  # [1,1,H,W]
        window = _2d_window.expand(channel, 1, window_size, window_size).contiguous()
        return window

    def forward(self, x, y):
        """
        x, y: tensores [B, C, H, W] en rango [0, data_range]
        Devuelve 1 - SSIM promedio en el batch.
        """
        if x.size(1) != self.channel or y.size(1) != self.channel:
            # Simplemente adaptamos el canal si es distinto (por si acaso)
            self.channel = x.size(1)
            self.window = self._create_window(self.window_size, self.sigma, self.channel).to(x.device)

        # Constantes de SSIM
        C1 = (self.K1 * self.data_range) ** 2
        C2 = (self.K2 * self.data_range) ** 2

        # Media local
        mu_x = torch.nn.functional.conv2d(
            x, self.window, padding=self.window_size // 2, groups=self.channel
        )
        mu_y = torch.nn.functional.conv2d(
            y, self.window, padding=self.window_size // 2, groups=self.channel
        )

        mu_x2 = mu_x * mu_x
        mu_y2 = mu_y * mu_y
        mu_xy = mu_x * mu_y

        # Varianzas y covarianza
        sigma_x2 = torch.nn.functional.conv2d(
            x * x, self.window, padding=self.window_size // 2, groups=self.channel
        ) - mu_x2
        sigma_y2 = torch.nn.functional.conv2d(
            y * y, self.window, padding=self.window_size // 2, groups=self.channel
        ) - mu_y2
        sigma_xy = torch.nn.functional.conv2d(
            x * y, self.window, padding=self.window_size // 2, groups=self.channel
        ) - mu_xy

        # Fórmula de SSIM
        num = (2 * mu_xy + C1) * (2 * sigma_xy + C2)
        den = (mu_x2 + mu_y2 + C1) * (sigma_x2 + sigma_y2 + C2)

        ssim_map = num / (den + 1e-8)
        ssim = ssim_map.mean()

        # Pérdida = 1 - SSIM promedio
        loss = 1 - ssim
        return loss


### Actualización de `build_loss` para incluir SSIM y SSIM+L1

Con la clase `SSIMLoss` definida, se extiende la función `build_loss` para
reconocer cuatro tipos de pérdida:

- `L1Loss`  → L1 estándar.
- `MSELoss` → L2 (MSE).
- `SSIM`    → `1 - SSIM(x, y)`.
- `SSIM_L1` → combinación lineal de SSIM y L1.


In [26]:
def build_loss(loss_cfg):
    """
    Construye la función de pérdida a partir de cfg.loss.

    Soporta:
      - L1Loss
      - MSELoss
      - SSIM
      - SSIM_L1 (combinación SSIM + L1)
    """
    loss_type = loss_cfg.type

    if loss_type == "L1Loss":
        return nn.L1Loss()

    elif loss_type == "MSELoss":
        return nn.MSELoss()

    elif loss_type == "SSIM":
        return SSIMLoss(
            window_size=loss_cfg.window_size,
            sigma=loss_cfg.sigma,
            data_range=loss_cfg.data_range,
            channel=3,  # nuestras imágenes son RGB
        )

    elif loss_type == "SSIM_L1":
        ssim_loss = SSIMLoss(
            window_size=loss_cfg.window_size,
            sigma=loss_cfg.sigma,
            data_range=loss_cfg.data_range,
            channel=3,
        )
        l1 = nn.L1Loss()
        l1_weight = loss_cfg.l1_weight

        class SSIML1Combined(nn.Module):
            def __init__(self, ssim_loss, l1, l1_weight):
                super().__init__()
                self.ssim_loss = ssim_loss
                self.l1 = l1
                self.l1_weight = l1_weight

            def forward(self, x, y):
                loss_ssim = self.ssim_loss(x, y)        # 1 - SSIM
                loss_l1 = self.l1(x, y)
                return loss_ssim + self.l1_weight * loss_l1

        return SSIML1Combined(ssim_loss, l1, l1_weight)

    else:
        raise NotImplementedError(f"Pérdida '{loss_type}' aún no implementada.")


### Configuración del modelo U-Net (Hydra)

Se requiere evaluar un autoencoder clásico y un autoencoder tipo **U-Net**.

Para permitir seleccionar esta arquitectura desde Hydra sin modificar el código,
se crea el archivo `conf/model/unet.yaml`, donde se definen sus parámetros:

- `in_channels`: número de canales de entrada (3 para RGB)
- `base_channels`: número inicial de filtros del encoder
- `depth`: cantidad de niveles de downsampling / upsampling
- `latent_dim`: tamaño del cuello (opcional)


In [27]:
Path("conf/model").mkdir(exist_ok=True)

unet_yaml = """in_channels: 3
base_channels: 32
depth: 4
latent_dim: 128
"""

with open("conf/model/unet.yaml", "w") as f:
    f.write(unet_yaml)

print("conf/model/unet.yaml creado")


conf/model/unet.yaml creado


### Implementación del Autoencoder U-Net

Este modelo sigue una estructura típica de U-Net:

1. **Encoder**:
   - Múltiples niveles de convoluciones + downsampling (stride 2).
   - Se almacenan características para las conexiones tipo "skip".

2. **Bottleneck**:
   - Capa completamente conectada para pasar al espacio latente.

3. **Decoder**:
   - ConvTransposed2D para upsampling simétrico.
   - Se concatenan los "skip connections" del encoder.

Este modelo debe:
- Usar la misma función de pérdida configurada en `cfg.loss`
- Usar el mismo optimizador configurado en `cfg.optimizer`
- Ser llamado desde Hydra con:
  `defaults: - model: unet`


In [28]:
class UNetEncoderBlock(nn.Module):
    def _init_(self, in_ch, out_ch):
        super()._init_()
        self.block = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.block(x)


class UNetDecoderBlock(nn.Module):
    def _init_(self, in_ch, out_ch):
        """
        in_ch: número de canales de entrada al bloque (antes del upsample)
        out_ch: número de canales de salida deseados después del bloque
        Este bloque realiza el upsample internamente (ConvTranspose2d) y luego
        procesa la concatenación con el skip connection.
        """
        super()._init_()
        # up transforma de in_ch -> out_ch (upsample)
        self.up = nn.ConvTranspose2d(in_ch, out_ch, kernel_size=2, stride=2)
        # luego se concatena con skip (out_ch + skip_ch), asumimos skip_ch == out_ch
        self.block = nn.Sequential(
            nn.Conv2d(out_ch * 2, out_ch, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.ReLU(),
        )

    def forward(self, x, skip):
        x = self.up(x)
        x = torch.cat([x, skip], dim=1)
        return self.block(x)


class LitUNetAutoencoder(pl.LightningModule):
    def _init_(self, model_cfg, loss_cfg, optimizer_cfg):
        super()._init_()
        self.save_hyperparameters(OmegaConf.to_container(model_cfg, resolve=True))

        self.model_cfg = model_cfg
        self.loss_cfg = loss_cfg
        self.optimizer_cfg = optimizer_cfg

        base = model_cfg.base_channels
        depth = model_cfg.depth
        in_ch = model_cfg.in_channels

        # ----- Encoder -----
        self.enc_blocks = nn.ModuleList()
        self.downsamples = nn.ModuleList()
        ch = in_ch
        channels = []

        for d in range(depth):
            out_ch = base * (2 ** d)
            self.enc_blocks.append(UNetEncoderBlock(ch, out_ch))
            channels.append(out_ch)
            ch = out_ch
            self.downsamples.append(nn.Conv2d(out_ch, out_ch, kernel_size=2, stride=2))

        # Bottleneck
        self.bottleneck = UNetEncoderBlock(ch, ch * 2)

        # ----- Decoder -----
        # Nota: cada UNetDecoderBlock ya hace su propio upsampling internamente.
        # Guardamos los bloques decoders en orden inverso.
        self.dec_blocks = nn.ModuleList()
        ch = ch * 2  # canales salientes del bottleneck

        for d in reversed(range(depth)):
            out_ch = base * (2 ** d)
            # dec block espera in_ch = ch (canales actuales antes de up), out_ch = canales deseados después
            self.dec_blocks.append(UNetDecoderBlock(ch, out_ch))
            ch = out_ch

        # Output (3 canales)
        self.final_conv = nn.Conv2d(base, 3, kernel_size=1)
        self.activation = nn.Sigmoid()

        # Loss
        self.criterion = build_loss(loss_cfg)

    def forward(self, x):
        skips = []
        out = x

        # Encoder
        for enc, down in zip(self.enc_blocks, self.downsamples):
            out = enc(out)
            skips.append(out)
            out = down(out)

        # Bottleneck
        out = self.bottleneck(out)

        # Decoder
        # IMPORTANTE: aquí NO hacemos up(out) ni cat manualmente porque cada dec_block
        # hace el up + concat internamente. Solo llamamos a cada dec_block con (out, skip).
        for dec, skip in zip(self.dec_blocks, reversed(skips)):
            out = dec(out, skip)

        out = self.final_conv(out)
        return self.activation(out)

    def _shared_step(self, batch, stage):
        x = batch
        x_hat = self(x)
        loss = self.criterion(x_hat, x)
        self.log(f"{stage}_loss", loss, prog_bar=True)
        return loss

    def training_step(self, batch, batch_idx):
        return self._shared_step(batch, "train")

    def validation_step(self, batch, batch_idx):
        return self._shared_step(batch, "val")

    def test_step(self, batch, batch_idx):
        return self._shared_step(batch, "test")

    def configure_optimizers(self):
        return build_optimizer(self.optimizer_cfg, self.parameters())

### Función de entrenamiento para soportar Autoencoder y U-Net

Se reescribe la función `train_autoencoder_with_hydra()` para:

- Cargar la configuración completa desde Hydra.
- Crear automáticamente el DataModule.
- Instanciar el modelo según `cfg.model`:
  - `autoencoder` → `LitAutoencoder`
  - `unet` → `LitUNetAutoencoder`
- Inicializar WandB.
- Crear un Trainer de Lightning.
- Entrenar el modelo.



In [ ]:
def train_autoencoder_with_hydra():
    """
    Versión actualizada: soporta modelos 'autoencoder' y 'unet'.
    Utiliza Hydra + Lightning + WandB para ejecutar entrenamientos reproducibles.
    """

    # 1. Cargar configuración completa desde Hydra
    with initialize(config_path="conf", version_base=None):
        cfg = compose(config_name="config")

    print("=========== CONFIGURACIÓN CARGADA ===========")
    print(OmegaConf.to_yaml(cfg))
    print("==============================================")

    # 2. Crear DataModule con los parámetros de cfg.data
    dm = MVTecDataModule(
        data_dir=cfg.data.data_dir,
        batch_size=cfg.data.batch_size,
        num_workers=cfg.data.num_workers,
        val_split=cfg.data.validation_split,
    )

    # 3. Instanciar modelo según cfg.model
    model_type = cfg.model._target_ if "_target_" in cfg.model else None

    # Detectar cuál modelo estamos usando
    if "unet" in cfg.model.__dict__['_content'] or "unet" in str(cfg.model):
        print("Instanciando modelo: U-Net Autoencoder")
        model = LitUNetAutoencoder(
            model_cfg=cfg.model,
            loss_cfg=cfg.loss,
            optimizer_cfg=cfg.optimizer,
        )
    else:
        print("Instanciando modelo: Autoencoder clásico")
        model = LitAutoencoder(
            model_cfg=cfg.model,
            loss_cfg=cfg.loss,
            optimizer_cfg=cfg.optimizer,
            image_size=cfg.data.image_size,
        )

    # 4. WandB Logger
    wandb_logger = WandbLogger(
        project=cfg.logger.project,
        entity=cfg.logger.entity,
        log_model=cfg.logger.log_model,
    )

    # 5. Trainer de Lightning con parámetros de Hydra
    trainer = pl.Trainer(
        max_epochs=cfg.trainer.max_epochs,
        log_every_n_steps=cfg.trainer.log_every_n_steps,
        deterministic=cfg.trainer.deterministic,
        enable_model_summary=cfg.trainer.enable_model_summary,
        enable_progress_bar=cfg.trainer.enable_progress_bar,
        logger=wandb_logger,
    )

    # 6. Entrenamiento
    trainer.fit(model, datamodule=dm)

    print("Entrenamiento finalizado correctamente")

    return model, dm, cfg


## Experimentación con WandB(Weight and Biases)

#### Proposito
      Este callback de PyTorch Lightning captura las pérdidas durante el entrenamiento y las visualiza al final

#### Hooks utilizados:
- on_train_epoch_end: Captura train_loss después de   cada época
- on_validation_epoch_end: Captura val_loss después de cada validación
- on_fit_end: Crea una gráfica matplotlib y la sube a Weights & Biases


#### Manejo de errores: 
Intenta .item() primero, luego float() para compatibilidad con diferentes versiones de PyTorch

In [30]:
class LossPlotCallback(pl.Callback):
    """Callback para graficar train/val loss al final del entrenamiento"""
    
    def __init__(self):
        super().__init__()
        self.train_losses = []
        self.val_losses = []
    
    def on_train_epoch_end(self, trainer, pl_module):
        """Guarda el train loss de cada epoch"""
        metrics = trainer.callback_metrics
        if 'train_loss' in metrics:
            try:
                self.train_losses.append(metrics['train_loss'].item())
            except Exception:
                self.train_losses.append(float(metrics['train_loss']))
    
    def on_validation_epoch_end(self, trainer, pl_module):
        """Guarda el val loss de cada epoch"""
        metrics = trainer.callback_metrics
        if 'val_loss' in metrics:
            try:
                self.val_losses.append(metrics['val_loss'].item())
            except Exception:
                self.val_losses.append(float(metrics['val_loss']))
    
    def on_fit_end(self, trainer, pl_module):
        """Grafica y loguea al final del entrenamiento"""
        if len(self.train_losses) == 0 or len(self.val_losses) == 0:
            return
        
        import matplotlib.pyplot as plt
        
        fig, ax = plt.subplots(figsize=(10, 6))
        epochs = range(1, len(self.train_losses) + 1)
        
        ax.plot(epochs, self.train_losses, label='Train Loss', marker='o', linewidth=2)
        ax.plot(epochs, self.val_losses, label='Val Loss', marker='s', linewidth=2)
        
        ax.set_xlabel('Epoch', fontsize=12)
        ax.set_ylabel('Loss', fontsize=12)
        ax.set_title('Training and Validation Loss', fontsize=14)
        ax.legend(fontsize=11)
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        
        # Loguear en WandB
        wandb.log({"loss_curve": wandb.Image(fig)})
        plt.close(fig)

## Callback para reconstrucción de imágenes

### Explicación:

- Propósito: Loguea comparaciones lado-a-lado de imágenes originales vs reconstruidas durante validación
- Estrategia de selección: Intenta elegir 1 imagen de cada tipo de producto MVTec (cable, capsule, screw, transistor) para mostrar diversidad

### Proceso:
- Obtiene un batch del validation loader
- Intenta seleccionar muestras diversas por categoría de producto
- Reconstruye las imágenes usando el modelo (sin gradientes)
- Concatena original y reconstrucción horizontalmente
- Sube las imágenes a WandB


In [ ]:
class ImageReconstructionLogger(pl.Callback):
    """Callback para loguear reconstrucciones de imágenes (elige muestras diversas por dataset)."""

    def __init__(self, num_images=4):
        super().__init__()
        self.num_images = num_images
        self.logged_count = 0

    def on_validation_epoch_end(self, trainer, pl_module):
        try:
            # Intentar obtener directamente un batch rápido (por compatibilidad)
            val_loader = trainer.datamodule.val_dataloader()
            batch = next(iter(val_loader))

            if isinstance(batch, (list, tuple)):
                batch_data = batch[0]
            else:
                batch_data = batch

            # DEBUG: mostrar forma del batch
            try:
                print(f"Batch shape: {getattr(batch_data, 'shape', None)}, necesito {self.num_images}")
            except Exception:
                pass

            # Intentar seleccionar muestras deterministas y diversas (una por dataset si es posible)
            try:
                val_subset = trainer.datamodule.val_set
                if val_subset is not None and hasattr(val_subset, "dataset"):
                    ds = val_subset.dataset  # MVTecDataset (full dataset antes del split)
                    indices = getattr(val_subset, "indices", None) or getattr(val_subset, "_indices", None)
                    if indices is not None and len(indices) > 0:
                        # Intentar escoger una muestra por cada dataset (cable, capsule, screw, transistor)
                        datasets_names = ['cable', 'capsule', 'screw', 'transistor']
                        chosen = []
                        seen = set()
                        for idx in indices:
                            name = ds.image_paths[idx].name.lower()
                            for d in datasets_names:
                                if d in name and d not in seen:
                                    chosen.append(idx)
                                    seen.add(d)
                                    break
                            if len(chosen) >= self.num_images:
                                break
                        # Si no conseguimos diversidad suficiente, usar los primeros índices del subset
                        if len(chosen) < self.num_images:
                            chosen = list(indices[: self.num_images])
                        sample_idx = chosen[: self.num_images]
                        sample_paths = [str(ds.image_paths[i]) for i in sample_idx]
                        print(" Rutas de validación (muestras que se usarán):", sample_paths)

                        # Construir batch_data a partir de los paths seleccionados para evitar confusiones
                        batch_tensors = []
                        for p in sample_paths:
                            img = Image.open(p).convert("RGB")
                            if hasattr(ds, "transform") and ds.transform is not None:
                                t = ds.transform(img)
                            else:
                                t = transforms.ToTensor()(img)
                            batch_tensors.append(t.unsqueeze(0))
                        if len(batch_tensors) > 0:
                            batch_data = torch.cat(batch_tensors, dim=0).to(pl_module.device)
            except Exception as e:
                print(f"Warning selection diversity failed: {e}")

            # Limitar a num_images (por si vino de val_loader)
            batch_data = batch_data[: self.num_images].to(pl_module.device)

            with torch.no_grad():
                reconstructed = pl_module(batch_data)

            # Pasar a CPU / numpy y crear visualización
            batch_cpu = batch_data.cpu()
            reconstructed_cpu = reconstructed.cpu()

            original_imgs = batch_cpu.numpy().transpose(0, 2, 3, 1)
            reconstructed_imgs = reconstructed_cpu.numpy().transpose(0, 2, 3, 1)

            images_to_log = []
            for i in range(len(original_imgs)):
                original = original_imgs[i]
                recon = reconstructed_imgs[i]
                combined = np.concatenate([original, recon], axis=1)
                combined = np.clip(combined, 0, 1)
                images_to_log.append(wandb.Image(combined, caption=f"Original vs Reconstructed {i}"))

            if len(images_to_log) > 0:
                self.logged_count += len(images_to_log)
                print(f"Logueadas {len(images_to_log)} imágenes. Total: {self.logged_count}")
                wandb.log({"reconstructions": images_to_log}, step=trainer.global_step)

        except Exception as e:
            print(f"Error en ImageReconstructionLogger: {type(e).__name__}: {e}")
            import traceback
            traceback.print_exc()

## Visualizador del espacio latente (PCA)

### Explicación:

Propósito: Visualiza el espacio latente del autoencoder reducido a 2D usando PCA cada 5 épocas

### Proceso:

- Extrae vectores latentes (z) de ~100 imágenes del validation set
- Aplica PCA para reducir dimensionalidad a 2D
- Crea scatter plot coloreado secuencialmente
- Muestra varianza explicada por cada componente principal

Compatibilidad: Verifica que el modelo tenga la estructura esperada (encoder, flatten, fc_mu)

In [34]:
class LatentSpaceVisualizer(pl.Callback):
    """Callback para visualizar el espacio latente con PCA"""
    
    def __init__(self, num_images=100):
        super().__init__()
        self.num_images = num_images
    
    def _get_latent(self, pl_module, batch):
        """Extrae vector latente para Autoencoder y UNet."""

        # Caso 1: Autoencoder clásico
        if hasattr(pl_module, "encoder") and hasattr(pl_module, "fc_mu"):
            x_enc = pl_module.encoder(batch)
            x_flat = pl_module.flatten(x_enc)
            z = pl_module.fc_mu(x_flat)
            return z

        # Caso 2: U-Net Autoencoder (usa bottleneck)
        if hasattr(pl_module, "bottleneck"):
            out = batch
            # Pasar por encoder UNet
            for enc, down in zip(pl_module.enc_blocks, pl_module.downsamples):
                out = enc(out)
                out = down(out)

            # Bottleneck real
            out = pl_module.bottleneck(out)

            # Aplanar (vector)
            z = out.view(out.size(0), -1)
            return z

        raise AttributeError("El modelo no tiene encoder ni bottleneck.")

    def on_validation_epoch_end(self, trainer, pl_module):
        """Visualiza el espacio latente cada N epochs"""
        if trainer.current_epoch % 5 != 0:
            return
        
        try:
            from sklearn.decomposition import PCA
            import matplotlib.pyplot as plt  # ← IMPORTAR AQUI TAMBIÉN
        except ImportError as e:
            print(f"Error de importación: {e}")
            return
        
        val_loader = trainer.datamodule.val_dataloader()
        latent_vectors = []
        
        with torch.no_grad():
            for batch_idx, batch in enumerate(val_loader):
                if batch_idx * len(batch) >= self.num_images:
                    break
                
                batch = batch.to(pl_module.device)
                
                try:
                    z = self._get_latent(pl_module, batch)
                    latent_vectors.append(z.cpu().numpy())
                except AttributeError as e:
                    print(f"⚠️ Modelo incompatible: {e}")
                    return
        
        if len(latent_vectors) == 0:
            print("No hay vectores latentes")
            return
        
        latent_vectors = np.concatenate(latent_vectors, axis=0)
        
        print(f"Aplicando PCA a {len(latent_vectors)} muestras...")
        try:
            pca = PCA(n_components=2, random_state=42)
            latent_2d = pca.fit_transform(latent_vectors)
            
            fig = self._create_plot(latent_2d, pca)
            wandb.log({"latent_space": wandb.Image(fig)}, step=trainer.global_step)
            plt.close(fig)
            
        except Exception as e:
            print(f"⚠️ Error en PCA: {e}")
    
    @staticmethod
    def _create_plot(latent_2d, pca):
        """Crea figura con PCA"""
        import matplotlib.pyplot as plt
        
        fig, ax = plt.subplots(figsize=(10, 8))
        scatter = ax.scatter(
            latent_2d[:, 0], 
            latent_2d[:, 1], 
            c=range(len(latent_2d)), 
            cmap='viridis', 
            alpha=0.6, 
            s=50
        )
        ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%})')
        ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%})')
        ax.set_title('Latent Space Visualization (PCA)')
        plt.colorbar(scatter, ax=ax)
        plt.tight_layout()
        return fig

### Visualizador de detección de anomalías 

#### Explicación:
Propósito: Evalúa el modelo en el test set cada 5 épocas y visualiza la capacidad de detección de anomalías

#### Proceso:
- Carga dataset de test (contiene imágenes normales y anómalas)
- Selecciona muestras aleatorias y las reconstruye
- Calcula MSE (Mean Squared Error) por imagen
- Normaliza el error y lo convierte en mapa de calor
- Crea visualización de 3 paneles: Original | Reconstrucción | Error Map
- Clasifica como "Good" o "Anomaly" según la mediana del MSE


#### Utilidad

Permite monitorear si el modelo aprende a detectar anomalías por tener mayor error de reconstrucción

In [35]:
class AnomalyDetectionVisualizer(pl.Callback):
    """Callback para visualizar reconstrucciones de imágenes normales y anómalas"""
    
    def __init__(self, data_dir='DATASET_128x128', num_images=4):
        super().__init__()
        self.data_dir = data_dir
        self.num_images = num_images
        self.test_dataset = None
    
    def setup(self, trainer, pl_module, stage=None):
        """Configurar dataset de prueba"""
        if self.test_dataset is None:
            self.test_dataset = MVTecDataset(
                root_dir=self.data_dir,
                split='test',
                transform=transforms.ToTensor()
            )
    
    def on_validation_epoch_end(self, trainer, pl_module):
        """Visualiza reconstrucciones del set de prueba"""
        if trainer.current_epoch % 5 != 0:
            return
        
        if self.test_dataset is None or len(self.test_dataset) == 0:
            return
        
        try:
            # Obtener muestras aleatorias
            n_samples = min(self.num_images, len(self.test_dataset))
            indices = np.random.choice(len(self.test_dataset), n_samples, replace=False)
            test_images = []
            
            for idx in indices:
                test_images.append(self.test_dataset[idx])
            
            test_batch = torch.stack(test_images).to(pl_module.device)
            
            with torch.no_grad():
                reconstructed = pl_module(test_batch)
            
            # Calcular MSE
            mse = torch.mean((test_batch - reconstructed) ** 2, dim=[1, 2, 3])
            
            # Pasar a CPU
            test_batch_cpu = test_batch.cpu()
            reconstructed_cpu = reconstructed.cpu()
            
            # Convertir a numpy
            original_imgs = test_batch_cpu.numpy().transpose(0, 2, 3, 1)
            reconstructed_imgs = reconstructed_cpu.numpy().transpose(0, 2, 3, 1)
            error_maps = mse.cpu().numpy()
            
            # Crear visualizaciones
            images_to_log = []
            median_error = np.median(error_maps)
            
            for i in range(len(original_imgs)):
                original = original_imgs[i]
                recon = reconstructed_imgs[i]
                error = error_maps[i]
                
                # Normalizar error map a [0, 1]
                error_normalized = (error - error_maps.min()) / (error_maps.max() - error_maps.min() + 1e-8)
                error_map_3ch = np.stack([error_normalized] * 3, axis=-1)
                
                # Concatenar paneles
                combined = np.concatenate([original, recon, error_map_3ch], axis=1)
                combined = np.clip(combined, 0, 1)
                
                label = "Good" if error < median_error else "Anomaly"
                images_to_log.append(
                    wandb.Image(combined, caption=f"{label} - MSE: {error:.4f}")
                )
            
            if len(images_to_log) > 0:
                wandb.log({"test_reconstructions": images_to_log}, step=trainer.global_step)
                
        except Exception as e:
            print(f"Error en AnomalyDetectionVisualizer: {e}")

### Función principal de experimentación

#### Explicación:
Propósito: Función orquestadora que ejecuta un experimento completo de entrenamiento

#### Pasos:
- Detección de hardware: Verifica GPU disponible y muestra información
- Inicialización WandB: Crea run para tracking de experimento
- Configuración Hydra: Carga configs YAML composables (modelo, loss, optimizer)
- DataModule: Instancia el cargador de datos MVTec
-Modelo: Selecciona entre autoencoder clásico o U-Net según nombre
- Callbacks: Configura ModelCheckpoint, EarlyStopping y visualizadores personalizados
- Trainer: Configura PyTorch Lightning con GPU forzado
- Entrenamiento: trainer.fit()
- Evaluación: trainer.test() en test set
- Cleanup: Cierra WandB con finally

In [36]:
def run_experiment(config_name, model_name, loss_name, optimizer_name, run_name):
    """
    Ejecuta un experimento con la configuración especificada
    
    Args:
        config_name: nombre de la configuración base (default: 'config')
        model_name: nombre del modelo en conf/model/
        loss_name: nombre de la función de pérdida en conf/loss/
        optimizer_name: nombre del optimizador en conf/optimizer/
        run_name: nombre del run en WandB
    """
    
    # VERIFICA GPU DISPONIBLE
    if not torch.cuda.is_available():
        print("WARNING: No hay GPU disponible. Se usará CPU.")
        device = "cpu"
    else:
        device = "cuda"
        print(f"GPU detectada: {torch.cuda.get_device_name(0)}")
        print(f"Memoria: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    
    # Inicializar WandB
    wandb.init(
        project="ae_experiments",
        name=run_name,
        config={
            "model": model_name,
            "loss": loss_name,
            "optimizer": optimizer_name,
            "device": device,
        }
    )
    
    try:
        # Cargar configuración con Hydra
        with initialize(config_path="conf", version_base=None):
            cfg = compose(
                config_name=config_name,
                overrides=[
                    f"model={model_name}",
                    f"loss={loss_name}",
                    f"optimizer={optimizer_name}",
                ]
            )
        
        print(f"\n{'='*60}")
        print(f"Ejecutando experimento: {run_name}")
        print(f"{'='*60}")
        print(OmegaConf.to_yaml(cfg))
        
        # Crear DataModule
        dm = MVTecDataModule(
            data_dir=cfg.data.data_dir,
            batch_size=cfg.data.batch_size,
            num_workers=cfg.data.num_workers,
            val_split=cfg.data.validation_split,
        )
        
        # Instanciar modelo
        model_type = model_name.lower()
        if "unet" in model_type:
            print("Modelo: U-Net Autoencoder")
            model = LitUNetAutoencoder(
                model_cfg=cfg.model,
                loss_cfg=cfg.loss,
                optimizer_cfg=cfg.optimizer,
            )
        else:
            print("Modelo: Autoencoder Clásico")
            model = LitAutoencoder(
                model_cfg=cfg.model,
                loss_cfg=cfg.loss,
                optimizer_cfg=cfg.optimizer,
                image_size=cfg.data.image_size,
            )
        
        # Callbacks personalizados
        callbacks = [
            ModelCheckpoint(
                monitor='val_loss',
                mode='min',
                save_top_k=3,
                save_last=True,
                dirpath=f"checkpoints/{run_name}",
            ),
            EarlyStopping(
                monitor='val_loss',
                mode='min',
                patience=10,
                verbose=True,
            ),
            ImageReconstructionLogger(num_images=4),
            LatentSpaceVisualizer(num_images=100),
            AnomalyDetectionVisualizer(num_images=4),
        ]
        
        # Logger de WandB
        wandb_logger = WandbLogger(
            project="ae_experiments",
            name=run_name,
            log_model=True,
        )
        
        # TRAINER CON GPU FORZADO
        trainer = pl.Trainer(
            max_epochs=cfg.trainer.max_epochs,
            logger=wandb_logger,
            callbacks=callbacks,
            log_every_n_steps=cfg.trainer.log_every_n_steps,
            deterministic=cfg.trainer.deterministic,
            enable_model_summary=cfg.trainer.enable_model_summary,
            enable_progress_bar=cfg.trainer.enable_progress_bar,
            accelerator="gpu", 
            devices=1,
            precision=32,
        )
        
        # Entrenar
        print(f"\nIniciando entrenamiento en {device.upper()}...")
        trainer.fit(model, datamodule=dm)
        
        # Evaluar en test set
        print(f"\nEvaluando en test set...")
        trainer.test(model, datamodule=dm)
        
        # Loguear métricas finales
        wandb.log({
            "final_train_loss": trainer.callback_metrics.get('train_loss', 0),
            "final_val_loss": trainer.callback_metrics.get('val_loss', 0),
            "device": device,
        })
        
        print(f"\nExperimento completado: {run_name}")
    
    finally:
        wandb.finish()

### Función para comparación de múltiples experimentos

#### Explicación:

Propósito: Automatiza la comparación de funciones de pérdida para un modelo específico

#### Estrategia: 
Mantiene constantes todos los hiperparámetros (optimizador, arquitectura) y solo varía la función de pérdida

#### Proceso:
- Escanea el directorio conf/loss/ para encontrar YAMLs de pérdidas disponibles
- Itera sobre cada función de pérdida
- Ejecuta run_experiment() con optimizador fijo (adam_mid)
- Recopila resultados (éxitos/fallos)
- Genera reporte final


#### Robustez: 
Captura excepciones para que un fallo no detenga toda la comparación

In [ ]:

def run_all_experiments(model_name, num_loss_functions=4):
    """
    Ejecuta exactamente 4 entrenamientos del mismo modelo
    con diferentes funciones de pérdida y MISMOS hiperparámetros
    
    Args:
        model_name: 'autoencoder' o 'unet'
        num_loss_functions: número de funciones de pérdida a probar
    """
    
    from pathlib import Path
    
    # Obtener funciones de pérdida disponibles
    conf_path = Path("conf")
    losses = sorted([f.stem for f in (conf_path / "loss").glob("*.yaml")])[:num_loss_functions]
    
    print(f"\n{'='*70}")
    print(f"COMPARACIÓN: {model_name.upper()}")
    print(f"{'='*70}")
    print(f"Modelo: {model_name}")
    print(f"Pérdidas a probar: {losses}")
    print(f"Optimizador fijo: adam_mid (lr=1e-3)")
    print(f"{'='*70}\n")
    
    successful = 0
    failed = 0
    results = []
    
    for i, loss in enumerate(losses, 1):
        run_name = f"{model_name}_{loss}_comparison"
        
        print(f"\n[{i}/{len(losses)}] Entrenando {model_name} con {loss.upper()}...")
        
        try:
            run_experiment(
                config_name="config",
                model_name=model_name,
                loss_name=loss,
                optimizer_name="adam_mid", 
                run_name=run_name,
            )
            successful += 1
            results.append({
                "model": model_name,
                "loss": loss,
                "status": "Exitoso"
            })
        except Exception as e:
            print(f"Error: {str(e)}")
            failed += 1
            results.append({
                "model": model_name,
                "loss": loss,
                "status": f"{str(e)[:50]}"
            })
    
    print(f"\n{'='*70}")
    print(f"COMPARACIÓN COMPLETADA: {model_name}")
    print(f"{'='*70}")
    print(f"Exitosos: {successful}/{len(losses)}")
    print(f"Fallidos: {failed}/{len(losses)}")
    print(f"{'='*70}\n")
    
    return results
    


### Script principal

#### Explicación:

#### Propósito: 
Script ejecutable que corre todos los experimentos de manera secuencial

#### Experimentos ejecutados:

- Autoencoder clásico: Prueba 4 funciones de pérdida diferentes
- U-Net autoencoder: Prueba las mismas 4 funciones de pérdida

Total: 8 entrenamientos completos


Output:  Tabla resumen con estado de cada experimento

Utilidad: Permite comparar arquitecturas (Autoencoder vs U-Net) y funciones de pérdida en un solo run

In [39]:
if __name__ == "__main__":
    # Crear directorio para checkpoints
    Path("checkpoints").mkdir(exist_ok=True)
    
    print("\n" + "="*70)
    print("🚀 INICIANDO EXPERIMENTACIÓN CON AUTOENCODERS")
    print("="*70 + "\n")
    
    all_results = []

    # ② Entrenamientos con U-Net
    print("\nU-NET - Comparación de pérdidas")
    unet_results = run_all_experiments(model_name="unet", num_loss_functions=4)
    all_results.extend(unet_results)
    
    # ① Entrenamientos con Autoencoder
    print("AUTOENCODER - Comparación de pérdidas")
    ae_results = run_all_experiments(model_name="autoencoder_small", num_loss_functions=4)
    all_results.extend(ae_results)
    
    # ③ Resumen final
    print("\n" + "="*70)
    print("RESUMEN FINAL DE EXPERIMENTACIÓN")
    print("="*70)
    for result in all_results:
        print(f"  {result['model']:20} | {result['loss']:10} | {result['status']}")
    print("="*70 + "\n")


🚀 INICIANDO EXPERIMENTACIÓN CON AUTOENCODERS


U-NET - Comparación de pérdidas

🔬 COMPARACIÓN: UNET
Modelo: unet
Pérdidas a probar: ['l1', 'l2', 'ssim', 'ssim_l1']
Optimizador fijo: adam_mid (lr=1e-3)


[1/4] Entrenando unet con L1...
GPU detectada: NVIDIA GeForce RTX 3060
Memoria: 8.59 GB



Ejecutando experimento: unet_l1_comparison
model:
  in_channels: 3
  base_channels: 32
  depth: 4
  latent_dim: 128
trainer:
  max_epochs: 20
  gpus: 1
  precision: 32
  deterministic: true
  check_val_every_n_epoch: 1
  log_every_n_steps: 10
  enable_model_summary: true
  gradient_clip_val: 0.0
  enable_progress_bar: true
logger:
  project: ae_experiments
  entity: null
  log_model: false
  offline: false
  tags: []
loss:
  type: L1Loss
  name: L1
optimizer:
  name: adam_mid
  type: Adam
  lr: 0.001
  weight_decay: 0.0
  betas:
  - 0.9
  - 0.999
seed: 42
data:
  data_dir: DATASET_128x128
  image_size: 128
  batch_size: 32
  num_workers: 0
  validation_split: 0.15
  test_split: 0.15
callbacks:
  monitor: val/loss
  mode: min
  filename: '{epoch:02d}-{val/loss:.4f}'
  save_top_k: 3
experiment:
  name: default_experiment
  description: Default autoencoder experiment

Modelo: U-Net Autoencoder


Error: _DeviceDtypeModuleMixin.__init__() got an unexpected keyword argument 'model_cfg'

[2/4] Entrenando unet con L2...
GPU detectada: NVIDIA GeForce RTX 3060
Memoria: 8.59 GB



Ejecutando experimento: unet_l2_comparison
model:
  in_channels: 3
  base_channels: 32
  depth: 4
  latent_dim: 128
trainer:
  max_epochs: 20
  gpus: 1
  precision: 32
  deterministic: true
  check_val_every_n_epoch: 1
  log_every_n_steps: 10
  enable_model_summary: true
  gradient_clip_val: 0.0
  enable_progress_bar: true
logger:
  project: ae_experiments
  entity: null
  log_model: false
  offline: false
  tags: []
loss:
  type: MSELoss
  name: L2
optimizer:
  name: adam_mid
  type: Adam
  lr: 0.001
  weight_decay: 0.0
  betas:
  - 0.9
  - 0.999
seed: 42
data:
  data_dir: DATASET_128x128
  image_size: 128
  batch_size: 32
  num_workers: 0
  validation_split: 0.15
  test_split: 0.15
callbacks:
  monitor: val/loss
  mode: min
  filename: '{epoch:02d}-{val/loss:.4f}'
  save_top_k: 3
experiment:
  name: default_experiment
  description: Default autoencoder experiment

Modelo: U-Net Autoencoder


Error: _DeviceDtypeModuleMixin.__init__() got an unexpected keyword argument 'model_cfg'

[3/4] Entrenando unet con SSIM...
GPU detectada: NVIDIA GeForce RTX 3060
Memoria: 8.59 GB


...

e:\Scripts\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


SystemExit: 1

C:\Users\Daniel\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Análisis cualitativo de las imágenes reconstruidas

En esta sección comparamos visualmente las reconstrucciones obtenidas por:

- El **autoencoder pequeño** (`autoencoder_small`)
- La **U-Net** (`unet`)

bajo cuatro funciones de pérdida diferentes:

- **L1**
- **L2**
- **SSIM**
- **SSIM + L1**

La idea es observar qué tan bien cada modelo logra:

1. Reconstruir correctamente las regiones **normales** de la imagen.
2. Mantener o resaltar las **diferencias** en las regiones donde hay **defectos/anomalías** (zonas que queremos detectar).



### Clase: cable

#### Ejemplo de imagen cable con un error

<img src="cable/test/bent_wire/002.png" width="150">

#### Autoencoder pequeño

![cable – Autoencoder pequeño (L1)](Images/cable/autoencoder_small_l1_comparison_cable.png)

![cable – Autoencoder pequeño (L2)](Images/cable/autoencoder_small_l2_comparison_cable.png)

![cable – Autoencoder pequeño (SSIM)](Images/cable/autoencoder_small_ssim_comparison_cable.png)

![cable – Autoencoder pequeño (SSIM + L1)](Images/cable/autoencoder_small_ssim_l1_comparison_cable.png)

#### U-Net

![cable – U-Net (L1)](Images/cable/unet_l1_comparison_cable.png)

![cable – U-Net (L2)](Images/cable/unet_l2_comparison_cable.png)

![cable – U-Net (SSIM)](Images/cable/unet_ssim_comparison_cable.png)

![cable – U-Net (SSIM + L1)](Images/cable/unet_ssim_l1_comparison_cable.png)

**Analisis**

- En general, la **U-Net** produce reconstrucciones más nítidas que el autoencoder pequeño, con mejor definición en bordes y estructura del cable.
- El **autoencoder pequeño** tiende a generar reconstrucciones algo más borrosas, especialmente alrededor de las zonas donde hay defectos. Esto puede hacer que la diferencia entre imagen original y reconstruida sea menos clara.
- Las pérdidas basadas en **SSIM** (SSIM y SSIM + L1) favorecen la preservación de la estructura global del cable, mientras que L1/L2 se enfocan más en errores pixel a pixel.
- Visualmente, cuando hay defectos, las reconstrucciones que se ven “demasiado suaves” o “promediadas” tienden a ocultar parte de la anomalía, mientras que las reconstrucciones más fieles a las regiones sanas dejan más evidente el error en la zona defectuosa (mayor diferencia entre original y reconstruida).


### Clase: capsule

#### Ejemplo de una imagen de una cápsula con un error

<img src="capsule/test/crack/001.png" width="150">

#### Autoencoder pequeño

![capsule – Autoencoder pequeño (L1)](Images/capsule/autoencoder_small_l1_comparison_capsule.png)

![capsule – Autoencoder pequeño (L2)](Images/capsule/autoencoder_small_l2_comparison_capsule.png)

![capsule – Autoencoder pequeño (SSIM)](Images/capsule/autoencoder_small_ssim_comparison_capsule.png)

![capsule – Autoencoder pequeño (SSIM + L1)](Images/capsule/autoencoder_small_ssim_l1_comparison_capsule.png)

#### U-Net

![capsule – U-Net (L1)](Images/capsule/unet_l1_comparison_capsule.png)

![capsule – U-Net (L2)](Images/capsule/unet_l2_comparison_capsule.png)

![capsule – U-Net (SSIM)](Images/capsule/unet_ssim_comparison_capsule.png)

![capsule – U-Net (SSIM + L1)](Images/capsule/unet_ssim_l1_comparison_capsule.png)

**Analisis**

- En las cápsulas es importante mantener la forma, contorno y textura de la superficie.
- La **U-Net** suele conservar mejor las formas geométricas y los bordes, lo que facilita notar cuando una cápsula tiene daño o textura anómala.
- El **autoencoder pequeño** puede introducir más suavizado, lo que dificulta separar visualmente una cápsula sana de una defectuosa.
- Las variantes con **SSIM** tienden a respetar mejor el contraste y la estructura general de la cápsula, mientras que L1/L2 pueden producir reconstrucciones con menos ruido pero también menos detalle fino.


### Clase: screw

#### Ejemplo de una imagen de un tornillo con un error

<img src="screw/test/scratch_head/008.png" width="150">

#### Autoencoder pequeño

![screw – Autoencoder pequeño (L1)](Images/screw/autoencoder_small_l1_comparison_screw.png)

![screw – Autoencoder pequeño (L2)](Images/screw/autoencoder_small_l2_comparison_screw.png)

![screw – Autoencoder pequeño (SSIM)](Images/screw/autoencoder_small_ssim_comparison_screw.png)

![screw – Autoencoder pequeño (SSIM + L1)](Images/screw/autoencoder_small_ssim_l1_comparison_screw.png)

#### U-Net

![screw – U-Net (L1)](Images/screw/unet_l1_comparison_screw.png)

![screw – U-Net (L2)](Images/screw/unet_l2_comparison_screw.png)

![screw – U-Net (SSIM)](Images/screw/unet_ssim_comparison_screw.png)

![screw – U-Net (SSIM + L1)](Images/screw/unet_ssim_l1_comparison_screw.png)

**Analisis**

- En tornillos, los detalles de la rosca y la geometría metálica son importantes.
- La **U-Net** tiende a preservar mejor la forma de la rosca y los contornos metálicos, lo que ayuda a que los defectos resalten como diferencias claras entre original y reconstrucción.
- El **autoencoder pequeño** puede perder detalles finos de textura, haciendo que las partes defectuosas se vean menos destacadas.
- Otra observación típica es que las pérdidas **L2/SSIM** ayudan a reconstruir mejor el patrón repetitivo de la rosca, mientras que L1 puede producir bordes algo más “cortados” o menos suaves.


### Clase: transistor

#### Ejemplo de una imagen de un transistor con un error

<img src="transistor/test/damaged_case/001.png" width="150">

#### Autoencoder pequeño

![transistor – Autoencoder pequeño (L1)](Images/transistor/autoencoder_small_l1_comparison_transistor.png)

![transistor – Autoencoder pequeño (L2)](Images/transistor/autoencoder_small_l2_comparison_transistor.png)

![transistor – Autoencoder pequeño (SSIM)](Images/transistor/autoencoder_small_ssim_comparison_transistor.png)

![transistor – Autoencoder pequeño (SSIM + L1)](Images/transistor/autoencoder_small_ssim_l1_comparison_transistor.png)

#### U-Net

![transistor – U-Net (L1)](Images/transistor/unet_l1_comparison_transistor.png)

![transistor – U-Net (L2)](Images/transistor/unet_l2_comparison_transistor.png)

![transistor – U-Net (SSIM)](Images/transistor/unet_ssim_comparison_transistor.png)

![transistor – U-Net (SSIM + L1)](Images/transistor/unet_ssim_l1_comparison_transistor.png)

**Analisis**

- En transistores hay muchos detalles pequeños: bordes de componentes, pistas, uniones, etc.
- La **U-Net** suele manejar mejor estas estructuras locales complejas, reconstruyendo de forma más limpia las regiones sanas y dejando diferencias más claras donde hay anomalías.
- El **autoencoder pequeño**, al tener menos capacidad, tiende a promediar más la información en regiones con muchos detalles, lo que puede “difuminar” parte de los defectos.
- De nuevo, las pérdidas basadas en **SSIM** favorecen la coherencia estructural de la imagen, lo que es útil en este tipo de objetos con patrones finos.


### Conclusiones generales del análisis cualitativo

- En todas las clases (**cable, capsule, screw, transistor**) se observa un patrón similar:
  - La **U-Net** produce reconstrucciones más detalladas y estructuralmente coherentes.
  - El **autoencoder pequeño** genera imágenes más suaves y con menos detalle fino, especialmente en zonas complejas.
- Respecto a las funciones de pérdida:
  - Las pérdidas **L2** y **SSIM** suelen dar reconstrucciones muy limpias de las regiones sanas.
  - Las pérdidas con **L1** tienden a ser un poco más robustas a outliers, pero pueden introducir artefactos o menos suavidad.
  - Las combinaciones como **SSIM + L1** buscan un balance entre estructura global y penalización por error absoluto.
- Para detección de anomalías, lo que interesa no es solo que la reconstrucción sea “bonita”, sino que:
  1. Las partes **normales** se reconstruyan bien (error bajo).
  2. Las partes **anómalas** se reconstruyan mal (error alto), de forma que el mapa de error permita separar claramente sano vs defectuoso.

En la siguiente sección se puede complementar este análisis visual con métricas cuantitativas (por ejemplo, usando el `test_loss` de cada configuración) y discutir cómo se relaciona la calidad visual de las reconstrucciones con el desempeño numérico del modelo.


## Análisis cuantitativo de los resultados

Esta sección presenta un análisis cuantitativo integral del desempeño de los modelos entrenados. Se incluyen:

- Comparación de desempeño (test_loss)
- La evolución de la pérdida durante el entrenamiento y validación.
- Un análisis de convergencia para comparar estabilidad y velocidad de aprendizaje.
- Interpretación del espacio latente para ambas arquitecturas.
- Conclusiones sobre la capacidad de generalización y su relación con las funciones de pérdida.

## 1. Comparación de desempeño (test_loss)

Los siguientes valores corresponden al error final en el conjunto de prueba para cada combinación de modelo y función de pérdida. Un valor más bajo indica mejor reconstrucción de las regiones normales y, por lo tanto, mejor capacidad de diferenciar anomalías.

| Modelo                 | Función de pérdida | Test Loss     |
|------------------------|--------------------|----------------|
| unet                   | l2                 | 0.00054789     |
| autoencoder_small      | l2                 | 0.0040282      |
| unet                   | ssim               | 0.0051722      |
| unet                   | ssim_l1            | 0.0064004      |
| unet                   | l1                 | 0.019768       |
| autoencoder_small      | l1                 | 0.036579       |
| autoencoder_small      | ssim               | 0.16097        |
| autoencoder_small      | ssim_l1            | 0.16594        |

### Observaciones principales
- El mejor resultado lo obtiene **U-Net con pérdida L2**, con un error extremadamente bajo.
- El autoencoder pequeño obtiene su mejor resultado también con **L2**, pero se queda claramente atrás respecto a U-Net.
- Las funciones basadas en **SSIM** obtienen valores más altos debido a que su métrica no está alineada con la medida usada en test_loss (que normalmente es MSE o similar).
- El peor desempeño global corresponde a **autoencoder_small con SSIM y SSIM+L1**, lo cual coincide con las limitaciones visuales observadas.




## 2. Curvas de entrenamiento y validación

Las siguientes figuras muestran cómo evoluciona la pérdida durante el proceso de entrenamiento para todos los experimentos registrados. Incluyen promedios de las corridas cargadas y reflejan la estabilidad del proceso de optimización.

### Pérdida de entrenamiento
![train loss](wandb/lossFunction/train_loss.png)

### Pérdida de validación
![val loss](wandb/lossFunction/val_loss.png)

### Interpretación de las curvas

**1. Estabilidad del entrenamiento**
- Las curvas de entrenamiento tienden a ser más suaves que las curvas de validación, lo cual es esperado.
- U-Net en la mayoría de sus pérdidas converge de manera más estable que el autoencoder pequeño.
- El autoencoder pequeño presenta fluctuaciones más marcadas, reflejando su limitada capacidad representacional.

**2. Velocidad de convergencia**
- La pérdida **L2** muestra la convergencia más rápida y estable en ambos modelos.
- Las pérdidas basadas en SSIM convergen más lentamente y de manera menos monótona debido a que capturan estadísticas estructurales y no únicamente errores pixel a pixel.

**3. Diferencia entre training y validation loss**
- Para U-Net, la brecha entre ambas curvas es pequeña, lo que indica buena generalización.
- En el autoencoder pequeño, la brecha es mayor, lo que sugiere sobreajuste por capacidad limitada.


## 3. Interpretación del espacio latente

Las siguientes imágenes muestran cómo cada modelo representa la información comprimida antes de la fase de decodificación.

### Autoencoder – L1
![ae_l1_latent](wandb/latentSpace/autoencoder_l1_comparison.png)

### Autoencoder – L2
![ae_l2_latent](wandb/latentSpace/autoencoder_l2_comparison.png)

### Autoencoder – SSIM
![ae_ssim_latent](wandb/latentSpace/autoencoder_ssim_comparison.png)

### Autoencoder – SSIM + L1
![ae_ssim_l1_latent](wandb/latentSpace/autoencoder_ssim_l1_comparison.png)

### U-Net – L1
![unet_l1_latent](wandb/latentSpace/unet_l1_comparison.png)

### U-Net – L2
![unet_l2_latent](wandb/latentSpace/unet_l2_comparison.png)

### U-Net – SSIM
![unet_ssim_latent](wandb/latentSpace/unet_ssim_comparison.png)

### U-Net – SSIM + L1
![unet_ssim_l1_latent](wandb/latentSpace/unet_ssim_l1_comparison.png)

### Interpretación del espacio latente

**1. Autoencoder**
- El espacio latente es más difuso: representa la información de manera comprimida pero con pérdida notable.
- Los vectores latentes muestran colapsos de información en configuraciones con SSIM, lo cual explica los altos `test_loss`.
- Con L2, las representaciones son más compactas y separables, lo cual coincide con su mejor desempeño cuantitativo.

**2. U-Net**
- Aunque U-Net no tiene un “latente puro” como el autoencoder, sus activaciones internas muestran patrones mucho más estructurados.
- Se observa mayor diferenciación entre regiones normales y anómalas incluso antes de la decodificación.
- Todas las pérdidas producen representaciones más limpias que las del autoencoder pequeño.

**3. Consistencia entre latente y reconstrucción**
- Cuando el espacio latente está bien estructurado, la reconstrucción suele ser más precisa.
- Esto se observa claramente en U-Net con L2 y SSIM.
- Cuando el espacio latente es ruidoso (autoencoder + SSIM), la reconstrucción también se degrada.


## 4. Relación entre test_loss y las curvas de entrenamiento

El resumen comparativo de `test_loss`  muestra que:

- U-Net con L2 alcanza el mejor error numérico debido a su capacidad para conservar detalles.
- El autoencoder presenta mayor error tanto en entrenamiento como en prueba.
- Las pérdidas basadas en SSIM no buscan minimizar el error pixel a pixel, lo que explica sus valores altos en `test_loss`.

Al observar las gráficas:

- Los modelos con menor test_loss corresponden también a curvas más suaves y monotónicas.
- La pérdida de validación confirma que U-Net no solo aprende más rápido sino también más consistentemente.
- El autoencoder muestra tendencia a caer en mínimos pobres, especialmente con funciones de pérdida no alineadas con el criterio de evaluación.


## 5. Conclusiones cuantitativas

1. **U-Net con pérdida L2 es la configuración más efectiva**, mostrando:
   - La convergencia más estable.
   - Las representaciones latentes más diferenciadas.
   - El menor error final en validación y prueba.
   - La mejor consistencia entre entrenamiento y validación.

2. **Las funciones de pérdida basadas en SSIM generan resultados visualmente aceptables, pero no óptimos bajo la métrica de evaluación**, debido a que su enfoque es estructural, no pixel a pixel.

3. **El autoencoder pequeño muestra limitaciones significativas**, especialmente cuando se utilizan pérdidas como SSIM y SSIM + L1, lo que se refleja tanto en las gráficas como en los valores finales.

4. **La relación entre complejidad del modelo, función de pérdida y estabilidad de aprendizaje queda clara**:
   - Modelos más capaces toleran pérdidas más complejas.
   - Modelos más simples necesitan pérdidas estrictas como L2 para converger adecuadamente.

Este análisis cuantitativo complementa el análisis visual previo y juntos permiten concluir cuál modelo y función de pérdida son más adecuados para la tarea de reconstrucción y detección de anomalías.
